**第2章 Ibor金利スワップ**

In [1]:
import myUtil as mu   ; from myABBR import *
# 0.取引日(evaluate),決済日(reference)の設定
tradeDT  = jDT(2022,8,19) ; setEvDT(tradeDT)
settleDT = calJP.advance(tradeDT, Tp2, DD)

# 1.指標金利オブジェクト
curveHDL = ql.RelinkableYieldTermStructureHandle()  
tbrIX    = ql.Tibor(pdFreqSA, curveHDL)
# 2.ヘルパーとカーブオブジェクト
cHelper, tbParRATE = [], []
for knd, tnr, rt in \
        [('depo','6m',0.13636), ('swap','1y',0.15249), ('swap','18m',0.18742),
         ('swap','2y',0.20541), ('swap','3y',0.23156), ('swap','4y',0.25653),
         ('swap','5y',0.28528), ('swap','6y',0.32341), ('swap','7y',0.36591)]:
    if knd == 'depo': cHelper.append(ql.DepositRateHelper(mu.sqHDL(rt/100),tbrIX)) 
    if knd == 'swap': cHelper.append(ql.SwapRateHelper(   mu.sqHDL(rt/100), 
                                    pD(tnr), calJP, freqSA, mFLLW, dcA365, tbrIX))
    tbParRATE.append(rt/100)                                   # パーレート用リスト
    
curveOBJ = ql.PiecewiseLogLinearDiscount(Tp2, calJP, cHelper, dcA365)
curveHDL.linkTo(curveOBJ) ; curveOBJ.enableExtrapolation()
# checking
print('tradeDT:{}, referenceDate:{}'
                .format(tradeDT.ISO(),curveOBJ.referenceDate().ISO()))
pd.DataFrame(curveOBJ.nodes(), columns=['node','DF'])[:6]

tradeDT:2022-08-19, referenceDate:2022-08-23


,node,DF
0,"August 23rd, 2022",1.000000
1,"February 24th, 2023",0.999309
2,"August 23rd, 2023",0.998477
3,"February 26th, 2024",0.997170
4,"August 23rd, 2024",0.995896
5,"August 25th, 2025",0.993059


In [2]:
# タプル -> データフレーム
display( pd.DataFrame( 
                    ([1, 2, 3],
                     [4, 5, 6]) , columns=['A', 'B', 'C']) )
# 辞書型 -> データフレーム
display( pd.DataFrame(
                ({'A':1, 'B':2,'C':3},
                 {'A':4, 'B':5,'C':6}) ,    index=[0, 1] ) )
# dictコンストラクタ -> データフレーム
display( pd.DataFrame( dict(A=[1,4], B=[2,5], C=[3,6])   ) )

,A,B,C
0,1,2,3
1,4,5,6


,A,B,C
0,1,2,3
1,4,5,6


,A,B,C
0,1,2,3
1,4,5,6


In [3]:
display( pd.DataFrame(
    ({'A':1, 'B':1*2}, {'A':2, 'B':2*2}   )) )
display( pd.DataFrame(
     {'A':x, 'B':x*2} for x in range(1,3)  ) )

,A,B
0,1,2
1,2,4


,A,B
0,1,2
1,2,4


In [4]:
# TIBORカーブ
def makeTiborCurve(crvDATA):
    '''makeTiborCurve(crvDATA)->[tbrIX,tbCrvOBJ,tbCrvHDL,tbParRT]'''
  # 1.指標金利オブジェクト
    tbCrvHDL = ql.RelinkableYieldTermStructureHandle() 
    tbrIX    = ql.Tibor(pdFreqSA, tbCrvHDL)
  # 2. HelperとTONAカーブオブジェクト
    cHelper, tbParRT = [], []
    for knd, tnr, rt in crvDATA:
       if knd == 'depo': cHelper.append(ql.DepositRateHelper(sqHDL(rt/100),tbrIX)) 
       if knd == 'swap': cHelper.append(ql.SwapRateHelper(   sqHDL(rt/100),
                                     pD(tnr), calJP, freqSA, mFLLW, dcA365, tbrIX))
       tbParRT.append(rt/100)                            # パーレート用リスト
    tbCrvOBJ = ql.PiecewiseLogLinearDiscount(Tp2, calJP, cHelper, dcA365)
    tbCrvHDL.linkTo(tbCrvOBJ) ; tbCrvOBJ.enableExtrapolation()
    return [tbrIX, tbCrvOBJ, tbCrvHDL, tbParRT]  # 4つのオブジェクトを戻す  

In [5]:
from myABBR import * ; import myUtil as mu

tradeDT  = jDT(2022,8,19) ; setEvDT(tradeDT)
settleDT = calJP.advance(tradeDT, Tp2, DD)
# 1.イールドカーブ オブジェクト
crvDATA = [('depo','6m',0.13636), ('swap','1y',0.15249), ('swap','18m',0.18742),
           ('swap','2y',0.20541), ('swap','3y',0.23156), ('swap','4y',0.25653),
           ('swap','5y',0.28528), ('swap','6y',0.32341), ('swap','7y',0.36591)]
tbrIX, tbCrvOBJ, tbCrvHDL, tbParRATE = mu.makeTiborCurve(crvDATA)
print('checking curve DF: {1:.6f} at {0}'.format(*tbCrvOBJ.nodes()[5]))

# 2.スケジュール オブジェクト 
effDT, matDT = jDT(2022,8,23), jDT(2024,8,23)
fixSCD = ql.Schedule(effDT, matDT, pdFreqSA, calJP, mFLLW, mFLLW, dtGENb, EoMf)
fltSCD = ql.Schedule(effDT, matDT, pdFreqSA, calJP, mFLLW, mFLLW, dtGENb, EoMf)

# 3.スワップオブジェクトとエンジン
payRcv,                ntlAMT,     cpnRT,   sprdRT                 = \
ql.VanillaSwap.Payer, 10_000_000,  0.012 ,   0.0

swapOBJ = ql.VanillaSwap(payRcv, ntlAMT, fixSCD, cpnRT,         dcA365,
                                         fltSCD, tbrIX, sprdRT, dcA365)
swapENG = ql.DiscountingSwapEngine(tbCrvHDL)
swapOBJ.setPricingEngine(swapENG)

# 計算結果
print('tradeDT:{}, referenceDate:{}'
                .format(tradeDT.ISO(),tbCrvOBJ.referenceDate().ISO()))
pd.DataFrame([  
    ['固定レグ時価'    ,swapOBJ.legNPV(0)],   ['変動レグ時価',swapOBJ.legNPV(1)],
    ['スワップ時価 NPV',swapOBJ.NPV()],       ['フェアレート',swapOBJ.fairRate()],
    ['フェアスプレッド',swapOBJ.fairSpread()]         ], columns=['計算結果', '']) 

checking curve DF: 0.993059 at August 25th, 2025
tradeDT:2022-08-19, referenceDate:2022-08-23


,計算結果,
0,固定レグ時価,-239781.110343
1,変動レグ時価,41044.531563
2,スワップ時価 NPV,-198736.578780
3,フェアレート,0.002054
4,フェアスプレッド,0.009946


In [6]:
print(list(fixSCD))

[Date(23,8,2022), Date(24,2,2023), Date(23,8,2023), Date(26,2,2024), Date(23,8,2024)]


In [7]:
fixSCDmk = ql.MakeSchedule(effDT, matDT, pdFreqSA)
print(list(fixSCDmk))
fixSCDmk = ql.MakeSchedule(effDT, matDT, pdFreqSA, calendar=calJP)
print(list(fixSCDmk))

[Date(23,8,2022), Date(23,2,2023), Date(23,8,2023), Date(23,2,2024), Date(23,8,2024)]
[Date(23,8,2022), Date(24,2,2023), Date(23,8,2023), Date(26,2,2024), Date(23,8,2024)]


In [8]:
dfSWP = pd.DataFrame({
    'fixingDate': cpn.fixingDate().ISO(),
    'accruStart': cpn.accrualStartDate().ISO(),
    'accruEnd':   cpn.accrualEndDate().ISO(),
    'payDate':    cpn.date(),                             # Date class(No ISO form)
    'days':       dcA365.dayCount(cpn.accrualStartDate(),cpn.accrualEndDate()),
    'rate':       cpn.rate(),
    'spread':     cpn.spread(),
    'amount':     cpn.amount(),
    } for cpn in map(ql.as_floating_rate_coupon, swapOBJ.leg(1)))       #変動レグ=1
# 起算日追加
dfEFF = pd.DataFrame([{'payDate': swapOBJ.startDate()}], columns=dfSWP.columns)
dfSWP = pd.concat([dfEFF, dfSWP], ignore_index=True)
# ディスカウントファクター(DF)
psDF = [1.0                   for dt in dfSWP.payDate if dt <= settleDT] #past   DF
fuDF = [tbCrvOBJ.discount(dt) for dt in dfSWP.payDate if settleDT < dt ] #future DF
dfSWP = pd.concat([dfSWP, pd.DataFrame(psDF+fuDF, columns=['DF']) ], axis=1)
dfSWP.payDate = dfSWP.payDate.map(lambda x: x.ISO())                         # ISOへ

dfSWP.style.format({'rate':'{:.6%}','spread':'{:.2%}','amount':'{:,.2f}','DF':'{:.8f}'})

,fixingDate,accruStart,accruEnd,payDate,days,rate,spread,amount,DF
0,nan,nan,nan,2022-08-23,nan,nan%,nan%,nan,1.00000000
1,2022-08-19,2022-08-23,2023-02-24,2023-02-24,185.000000,0.136360%,0.00%,"6,911.40",0.99930934
2,2023-02-21,2023-02-24,2023-08-23,2023-08-23,180.000000,0.169082%,0.00%,"8,338.28",0.99847678
3,2023-08-21,2023-08-23,2024-02-26,2024-02-26,187.000000,0.255717%,0.00%,"13,101.12",0.99717037
4,2024-02-21,2024-02-26,2024-08-23,2024-08-23,179.000000,0.261022%,0.00%,"12,800.82",0.99589555


In [9]:
list(swapOBJ.leg(1))

[<QuantLib.QuantLib.CashFlow; proxy of <Swig Object of type 'ext::shared_ptr< CashFlow > *' at 0x000001DD02BCB780> >,
 <QuantLib.QuantLib.CashFlow; proxy of <Swig Object of type 'ext::shared_ptr< CashFlow > *' at 0x000001DD02BCA9D0> >,
 <QuantLib.QuantLib.CashFlow; proxy of <Swig Object of type 'ext::shared_ptr< CashFlow > *' at 0x000001DD02BCA9A0> >,
 <QuantLib.QuantLib.CashFlow; proxy of <Swig Object of type 'ext::shared_ptr< CashFlow > *' at 0x000001DD02BCB7B0> >]

In [10]:
list(swapOBJ.leg(1))[0].amount()

6911.397260274121

In [11]:
ql.as_floating_rate_coupon(swapOBJ.leg(1)[1]).rate()

0.0016908187880833778

In [12]:
help(swapOBJ.leg(1)[1])

Help on CashFlow in module QuantLib.QuantLib object:

class CashFlow(Observable)
 |  CashFlow(*args, **kwargs)
 |  
 |  Proxy of C++ CashFlow class.
 |  
 |  Method resolution order:
 |      CashFlow
 |      Observable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      __init__(Observable self) -> Observable
 |  
 |  __repr__ = _swig_repr(self)
 |  
 |  amount(self)
 |      amount(CashFlow self) -> Real
 |  
 |  date(self)
 |      date(CashFlow self) -> Date
 |  
 |  hasOccurred(self, *args)
 |      hasOccurred(CashFlow self, Date refDate=Date()) -> bool
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __swig_destroy__ = delete_CashFlow(...)
 |      delete_CashFlow(CashFlow self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  thisown
 |      The membership flag
 |  
 |  ----------------

In [13]:
help(ql.as_floating_rate_coupon(swapOBJ.leg(1)[1]))

Help on FloatingRateCoupon in module QuantLib.QuantLib object:

class FloatingRateCoupon(Coupon)
 |  FloatingRateCoupon(*args, **kwargs)
 |  
 |  Proxy of C++ FloatingRateCoupon class.
 |  
 |  Method resolution order:
 |      FloatingRateCoupon
 |      Coupon
 |      CashFlow
 |      Observable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      __init__(Observable self) -> Observable
 |  
 |  __repr__ = _swig_repr(self)
 |  
 |  adjustedFixing(self)
 |      adjustedFixing(FloatingRateCoupon self) -> Rate
 |  
 |  convexityAdjustment(self)
 |      convexityAdjustment(FloatingRateCoupon self) -> Rate
 |  
 |  fixingDate(self)
 |      fixingDate(FloatingRateCoupon self) -> Date
 |  
 |  fixingDays(self)
 |      fixingDays(FloatingRateCoupon self) -> Integer
 |  
 |  gearing(self)
 |      gearing(FloatingRateCoupon self) -> Real
 |  
 |  index(self)
 |      index(FloatingRateCoupon self) -> ext::shared_ptr< InterestRateIndex >
 |  
 |  

In [14]:
print('変動レグ時価(手計算): {:,.3f}'.format((dfSWP.amount*dfSWP.DF).sum()) )

変動レグ時価(手計算): 41,044.532


In [15]:
def swapCashFlow(swapOBJ, curveOBJ, leg=1, eff=0, dc=dcA365): 
    '''swapCashFlow(swapOBJ, curveOBJ, leg=1:FLoat  0:Fix)-> DataFrame)'''  
    if leg == 1:  # 変動ﾚｸﾞ leg(1)
        dfSWP = pd.DataFrame({
            'fixingDate': cpn.fixingDate().ISO(),
            'accruStart': cpn.accrualStartDate().ISO(),
            'accruEnd':   cpn.accrualEndDate().ISO(),
            'payDate':    cpn.date(),                            # No ISO form
            'days':       dc.dayCount(cpn.accrualStartDate(),cpn.accrualEndDate()),
            'rate':       cpn.rate(),
            'spread':     cpn.spread(),
            'amount':     cpn.amount(),
            } for cpn in map(ql.as_floating_rate_coupon, swapOBJ.leg(1)))
    else:          # 固定ﾚｸﾞ leg(0)
        dfSWP = pd.DataFrame({
            'nominal':    cpn.nominal(),
            'accruStart': cpn.accrualStartDate().ISO(),
            'accruEnd':   cpn.accrualEndDate().ISO(),
            'payDate':    cpn.date(),
            'days':       dc.dayCount(cpn.accrualStartDate(),cpn.accrualEndDate()),
            'rate':       cpn.rate(),
            'amount':     cpn.amount()
            } for cpn in map(ql.as_fixed_rate_coupon, swapOBJ.leg(0)))
    # 起算日の挿入
    dfEFF = pd.DataFrame([{'payDate': swapOBJ.startDate()}], columns=dfSWP.columns)
    dfSWP = pd.concat([dfEFF, dfSWP], ignore_index=True)                
    # ディスカウントファクター(DF)
    settleDT = curveOBJ.referenceDate()
    psDF = [1.0                   for dt in dfSWP.payDate if dt <= settleDT] # past   DF
    fuDF = [curveOBJ.discount(dt) for dt in dfSWP.payDate if settleDT < dt ] # future DF
    dfSWP = pd.concat([dfSWP, pd.DataFrame(psDF+fuDF, columns=['DF']) ], axis=1)
    dfSWP.payDate = dfSWP.payDate.map(lambda x: x.ISO())                        # ISOへ
    return dfSWP

In [16]:
tbrIX.addFixing(jDT(2022,8,19),0.13636/100,True)

In [17]:
# fixing日とfixingの例
prevPayDT = settleDT if swapOBJ.floatingSchedule().startDate() >= settleDT \
                   else swapOBJ.floatingSchedule().previousDate(settleDT)
fixedDT = calJP.advance(prevPayDT, -Tp2, DD)
tbrIX.addFixing(fixedDT,0.13636/100,True)

In [18]:
# 新しい日付設定
tradeDT  = jDT(2023,5,1) ; setEvDT(tradeDT)
settleDT = calJP.advance(tradeDT, Tp2, DD)

# fixing日とfixingの例
prevPayDT = settleDT if swapOBJ.floatingSchedule().startDate() >= settleDT \
                   else swapOBJ.floatingSchedule().previousDate(settleDT)
fixedDT = calJP.advance(prevPayDT, -Tp2, DD)
tbrIX.addFixing(fixedDT,0.1/100,True)

# float leg CF (leg=1)
dfFixCF = mu.swapCashFlow(swapOBJ, tbCrvOBJ, leg=1)
dfFixCF.style.format(fmtSCF)

,fixingDate,accruStart,accruEnd,payDate,days,rate,spread,amount,DF
0,nan,nan,nan,2022-08-23,nan,nan%,nan%,nan,1.00000000
1,2022-08-19,2022-08-23,2023-02-24,2023-02-24,185,0.136360%,0.000%,"6,911.40",1.00000000
2,2023-02-21,2023-02-24,2023-08-23,2023-08-23,180,0.100000%,0.000%,"4,931.51",0.99960048
3,2023-08-21,2023-08-23,2024-02-26,2024-02-26,187,0.155450%,0.000%,"7,964.13",0.99880502
4,2024-02-21,2024-02-26,2024-08-23,2024-08-23,179,0.221531%,0.000%,"10,864.14",0.99772108


In [19]:
# 過去のfixingデータの表示
[(dd.ISO(), vv) for dd, vv in zip(tbrIX.timeSeries().dates(), 
                                  tbrIX.timeSeries().values())]

[('2022-08-19', 0.0013636000000000002), ('2023-02-21', 0.001)]

In [20]:
# 日付を元へ戻す
tradeDT  = jDT(2022,8,19) ; setEvDT(tradeDT)
settleDT = calJP.advance(tradeDT, Tp2, DD)

In [21]:
# fixed leg CF (leg=0)
dfFixCF = mu.swapCashFlow(swapOBJ, tbCrvOBJ, leg=0)
display( dfFixCF.style.format(fmtSCF) )
print('アニュイティ:', (dfFixCF.days/365 * dfFixCF.DF).sum() )

,nominal,accruStart,accruEnd,payDate,days,rate,amount,DF
0,nan,nan,nan,2022-08-23,nan,nan%,nan,1.00000000
1,"10,000,000.00",2022-08-23,2023-02-24,2023-02-24,185,1.200000%,"60,821.92",0.99930934
2,"10,000,000.00",2023-02-24,2023-08-23,2023-08-23,180,1.200000%,"59,178.08",0.99847678
3,"10,000,000.00",2023-08-23,2024-02-26,2024-02-26,187,1.200000%,"61,479.45",0.99717037
4,"10,000,000.00",2024-02-26,2024-08-23,2024-08-23,179,1.200000%,"58,849.32",0.99589555


アニュイティ: 1.998175919523169


In [22]:
from myABBR import *
# 受渡日と満期日のスケジュールオブジェクト
settleDT, matDT = jDT(2023,8,23), jDT(2025,8,23) 
annSCD = ql.Schedule(settleDT, matDT, pdFreqSA, calJP, mFLLW, mFLLW, dtGENb, EoMf)

# アニュイティ計算
discFCT = nA([tbCrvOBJ.discount(xx) for xx in annSCD][1:]) 
tnrLST  = np.diff([dcA365.yearFraction(annSCD.startDate(),xx) for xx in annSCD]) 
annuity = np.sum(tnrLST * discFCT)

print('discount factors:', discFCT); print('tenor list      :', tnrLST, end=', ')
print('アニュイティ:{:.6f}'.format(annuity))

discount factors: [0.99717 0.9959  0.99446 0.99306]
tenor list      : [0.51233 0.49041 0.50959 0.49589], アニュイティ:1.998490


In [23]:
import xlwings as xw ; from myABBR import * ; import myUtil as mu

@xw.func 
@xw.arg('dateINF', ndim=1)
@xw.arg('legINF', ndim=1)
@xw.arg('crvDATA', ndim=2)
@xw.ret(index=False, header=True, expand='table')

def tiborSWAP(dateINF,legINF,crvDATA,cftable=-1): # cftable=> 固定表:0, 変動表:1
    # 0.引数の処理
    tradeDT, effDT,  matDT                             = dateINF
    payRcv,  ntlAMT, cpnRT, sprdRT, fixedIX            = legINF

    tradeDT,          effDT,      matDT,      payRcv   =\
    dDT(tradeDT), dDT(effDT), dDT(matDT), int(payRcv)

    settleDT = calJP.advance(tradeDT,Tp2,DD) ; setEvDT(tradeDT)
    # 1.カーブ作成
    tbrIX, tbCrvOBJ, tbCrvHDL, tbParRATE = mu.makeTiborCurve(crvDATA)
    # 2.スケジュールオブジェクト 
    fixSCD = ql.Schedule(effDT, matDT, pdFreqSA, calJP, mFLLW, mFLLW, dtGENb, EoMf)
    # 3.スワップオブジェクト
    swapOBJ = ql.VanillaSwap(payRcv,ntlAMT,fixSCD,cpnRT,       dcA365,
                                           fixSCD,tbrIX,sprdRT,dcA365)
    swapENG = ql.DiscountingSwapEngine(tbCrvHDL); swapOBJ.setPricingEngine(swapENG)
    # 4.fixed指標金利の設定
    prevPayDT = settleDT if swapOBJ.floatingSchedule().startDate() >= settleDT \
                       else swapOBJ.floatingSchedule().previousDate(settleDT)
    fixedDT = calJP.advance(prevPayDT,-Tp2,DD)
    tbrIX.clearFixings() ;  tbrIX.addFixing(fixedDT,fixedIX)
    # 5.Retrun   ( cftable=> 0:固定表, 1:変動表 他:評価 )
    if cftable == 1 :
        df = mu.swapCashFlow(swapOBJ, tbCrvOBJ, 1)
    elif cftable == 0 :
        df = mu.swapCashFlow(swapOBJ, tbCrvOBJ, 0)
    else: 
        df =pd.DataFrame([  
        ['固定レグ時価'    ,swapOBJ.legNPV(0)],   ['変動レグ時価',swapOBJ.legNPV(1)],
        ['スワップ時価 NPV',swapOBJ.NPV()],       ['フェアレート',swapOBJ.fairRate()],
        ['フェアスプレッド',swapOBJ.fairSpread()]         ], columns=['計算結果', ''])
    return df